In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


import re

print("Tensorflow Version",tf.__version__)

In [ ]:
data = pd.read_csv('../input/spam-mails-dataset/spam_ham_dataset.csv')
data.head()

In [ ]:
data = data.drop(['Unnamed: 0', 'label'], axis=1)

In [ ]:
val_count = data.label_num.value_counts()

plt.figure(figsize=(8,4))
plt.bar(val_count.index, val_count.values)
plt.title("Spam/ham Data Distribution")

In [ ]:
data.head()

In [ ]:
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S+|[^A-Za-z0-9]:\S+|subject:\S+|nbsp"

In [ ]:
def preprocess(text, stem=False):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)
data.text = data.text.apply(lambda x: preprocess(x))
data.head()

In [ ]:
x = data['text']
y = data['label_num']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                         random_state=7) # Splits Dataset into Training and Testing set
print("Train Data size:", len(x_train))
print("Test Data size", len(x_test))

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1000
print("Vocabulary Size :", vocab_size)

In [ ]:
x_train.head().reset_index()

In [ ]:
word_index

In [ ]:
from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(x_train),
                        maxlen = 50)
x_test = pad_sequences(tokenizer.texts_to_sequences(x_test),
                       maxlen = 50)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

In [ ]:
x_train[0]

In [ ]:
type(y_train)

In [ ]:
x_test[0].shape

# ANN


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
ann = Sequential()
ann.add(Dense(16, activation='relu', input_dim=50))
# Adding dropout to prevent overfitting
ann.add(Dropout(0.1))
ann.add(Dense(1, activation='sigmoid'))

In [ ]:
ann.summary()

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
ann.fit(x_train, y_train, batch_size=100, epochs=150)

In [ ]:
# Predicting the Test set results
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)

In [ ]:
y_pred[0]

In [ ]:
y_test1 = np.array(y_test)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

In [ ]:
695 +43 +222+75

In [ ]:
print("Our accuracy is {}%".format(((cm[0][0] + cm[1][1])/1035)*100))

# RNN 

In [ ]:
from keras.layers import SimpleRNN
rnn = Sequential()
rnn.add(SimpleRNN(128, activation='relu', input_dim=50 , return_sequences = True))
rnn.add(SimpleRNN(64, activation='relu' ,  return_sequences = False))
# Adding dropout to prevent overfitting
#rnn.add(Dropout(0.1))
rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
rnn.summary()

In [ ]:
rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
#x_train1 = x_train.reshape(4136,1,50)
y_train1 = np.array(y_train).reshape(4136,1,1)

In [ ]:
rnn.fit(x_train1, y_train1, batch_size=100, epochs=150)

# lstm

In [ ]:
#LSTM hyperparameters
MAX_SEQUENCE_LENGTH = 50
n_lstm = 200
drop_lstm =0.2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional
embeding_dim = 16
drop_value = 0.2
n_dense = 24
num_epochs = 5
early_stop = EarlyStopping(monitor='val_loss', patience=3)
#LSTM Spam detection architecture
model1 = Sequential()
model1.add(Embedding(vocab_size, embeding_dim, input_length=MAX_SEQUENCE_LENGTH))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(Dense(1, activation='relu'))
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = model1.fit(x_train, y_train, epochs=num_epochs, verbose=2)

In [ ]:
# Predicting the Test set results
y_pred_lstm = model1.predict(x_test)
y_pred_lstm = (y_pred_lstm > 0.5)

In [ ]:
def func(x):
    if x>0.5:
        return 1
    else:
        return 0

In [ ]:
y_pred_lstm1 = y_pred_lstm.reshape(len(y_pred_lstm[0]))
for j in range (len(y_pred_lstm1)):
    y_pred_lstm1[j] = func(y_pred_lstm1[j])
    #for i in range (len(y_pred1)):
    pred = int(y_pred_lstm1.all())

In [ ]:
len(y_pred_lstm[0])

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test, y_pred_lstm)

In [ ]:
cm1

In [ ]:
print("Our accuracy is {}%".format(((cm1[0][0] + cm1[1][1])/1035)*100))

# bi lstm

In [ ]:
model2 = Sequential()
model2.add(Embedding(vocab_size, embeding_dim, input_length=MAX_SEQUENCE_LENGTH))
model2.add(Bidirectional(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True)))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
# Training
num_epochs = 5
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = model2.fit(x_train, y_train, epochs=num_epochs, 
                    callbacks =[early_stop], verbose=2)

In [ ]:
y_pred_bi_lstm = model2.predict(x_test)
y_pred_bi_lstm = (y_pred_bi_lstm > 0.5)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(y_test, y_pred_bi_lstm)

In [ ]:
cm2

In [ ]:
print("Our accuracy is {}%".format(((cm2[0][0] + cm2[1][1])/1035)*100))